In [1]:
import json
import csv
import joblib
from os import path
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

pd.set_option('display.max_columns', 500)

In [2]:
#ds = datetime.strptime('2017-01-01', '%Y-%m-%d')
df=pd.read_csv('data/to_train.csv')
df['marketTime']=pd.to_datetime(df['marketTime'])
df.sort_values(by='marketTime', inplace=True)
df['rid']=df.course.str.lower().replace(regex=True,to_replace=r'\\W|\s',value=r'')+df.marketTime.dt.strftime('%Y%m%d%H%M')
df=df.fillna(0)

df_pred=pd.read_csv('data/topred.csv')
df_pred=df_pred.fillna(0)


In [3]:
def get_matrix(dfx,means,stds, enc):
    x_num = (dfx[cols_numerical].values - means) / stds
    x_cat=enc.transform(dfx[cols_categorical]).toarray()
    return np.hstack([x_num,x_cat])

In [4]:
cols=['rid','course', 'marketTime', 'horseName', 'position','res_win', 'res_place', 'runners', 'ncond', 'metric', 'class','decimalPrice', 'age', 'RPR', 'TR', 'OR', 'weight','age_rank', 'decimalPrice_rank','weight_rank', 'RPR_rank', 'TR_rank','OR_rank', 'res_win_h_avg_rank', 'res_place_h_avg_rank','decimalPrice_diff_h_avg_rank', 'position_diff_h_avg_rank','res_win_t_avg_rank', 'res_place_t_avg_rank','decimalPrice_diff_t_avg_rank', 'position_diff_t_avg_rank','res_win_j_avg_rank', 'res_place_j_avg_rank','decimalPrice_diff_j_avg_rank', 'position_diff_j_avg_rank','metric_h_avg', 'res_win_h_avg','res_place_h_avg','decimalPrice_diff_h_avg', 'RPR_diff_h_avg', 'TR_diff_h_avg','OR_diff_h_avg', 'position_diff_h_avg', 'metric_t_avg', 'res_win_t_avg','res_place_t_avg', 'decimalPrice_diff_t_avg', 'RPR_diff_t_avg','TR_diff_t_avg', 'OR_diff_t_avg', 'position_diff_t_avg', 'metric_j_avg','res_win_j_avg', 'res_place_j_avg', 'decimalPrice_diff_j_avg','RPR_diff_j_avg', 'TR_diff_j_avg', 'OR_diff_j_avg','position_diff_j_avg', 'res_win_h_avg_diff', 'res_place_h_avg_diff','decimalPrice_diff_h_avg_diff', 'position_diff_h_avg_diff','res_win_t_avg_diff', 'res_place_t_avg_diff','decimalPrice_diff_t_avg_diff', 'position_diff_t_avg_diff','res_win_j_avg_diff', 'res_place_j_avg_diff','decimalPrice_diff_j_avg_diff', 'position_diff_j_avg_diff', 'age_diff','decimalPrice_diff', 'weight_diff', 'RPR_diff', 'TR_diff', 'OR_diff']

df=df[cols]
# Get all RIDs
rids=df.rid.unique()
validate=rids[-10000:]
rids=rids[:-10000]
np.random.shuffle(rids)

# Get three RIDs lists
#train, test, validate = np.split(rids, [int(.7*len(rids)), int(.85*len(rids))]) 
train, test = np.split(rids, [int(.9*len(rids))]) 
# Split dataframe on parts
train_df=df[df['rid'].isin(train)]
test_df=df[df['rid'].isin(test)]
validate_df=df[df['rid'].isin(validate)]

# Catogorical are columns with rank and three others
cols_categorical=[col for col in df.columns if '_rank' in col]+['ncond', 'class']
#cols_categorical=['ncond', 'class']

# Filter out cols_categorical
cols=[col for col in df.columns if not col in cols_categorical]
# Numerical are columns with avg and diff
cols_numerical=[col for col in cols if ('_avg' in col) or ('_diff' in col)]
#cols_numerical=[col for col in cols if ('_avg' in col)]
# ...and some others
#cols_numerical=cols_numerical+['RPR', 'TR', 'OR', 'win_drift']
cols_numerical=cols_numerical+ ['RPR', 'TR','OR','decimalPrice']

# The rest columns 
cols=[col for col in cols if not col in cols_numerical]

cols_descr=['rid','course','marketTime', 'horseName', 'runners','decimalPrice','res_win', 'res_place']

# Prepare data
X_train=train_df[cols_numerical].values
means = X_train.mean(axis=0, keepdims=True)
stds = X_train.std(axis=0, keepdims=True) + 1e-10
enc = OneHotEncoder(handle_unknown='ignore')
enc.fit(train_df[cols_categorical])

# Convert data
train_x=get_matrix(train_df,means,stds, enc)
train_y=train_df['res_win'].astype(int).values

test_x=get_matrix(test_df,means,stds, enc)
test_y=test_df['res_win'].astype(int).values

validate_x=get_matrix(validate_df,means,stds, enc)
validate_y=validate_df['res_win'].astype(int).values

In [24]:
df_pred=pd.read_csv('data/topred.csv')
df_pred=df_pred.fillna(0)
df_pred=df_pred[['date']+cols]
pred_x=get_matrix(df_pred,means,stds, enc)


# TF

train_y=train_y.reshape(-1,1)
train_y=np.hstack((train_y, 1-train_y))
test_y=test_y.reshape(-1,1)
test_y=np.hstack((test_y, 1-test_y))
validate_y=validate_y.reshape(-1,1)
validate_y=np.hstack((validate_y, 1-validate_y))

In [16]:
def get_batch(i):
    start_index=i*batch_size
    end_index=start_index+batch_size
    if end_index>=len(train_x):
        end_index=len(train_x)-1
    return train_x[start_index:end_index],train_y[start_index:end_index]

def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

def shuffle_batch(X, y, batch_size):
    rnd_idx = np.random.permutation(len(X))
    n_batches = len(X) // batch_size
    for batch_idx in np.array_split(rnd_idx, n_batches):
        X_batch, y_batch = X[batch_idx], y[batch_idx]
        yield X_batch, y_batch

from scipy.special import erfc

# alpha and scale to self normalize with mean 0 and standard deviation 1
# (see equation 14 in the paper):
alpha_0_1 = -np.sqrt(2 / np.pi) / (erfc(1/np.sqrt(2)) * np.exp(1/2) - 1)
scale_0_1 = (1 - erfc(1 / np.sqrt(2)) * np.sqrt(np.e)) * np.sqrt(2 * np.pi) * (2 * erfc(np.sqrt(2))*np.e**2 + np.pi*erfc(1/np.sqrt(2))**2*np.e - 2*(2+np.pi)*erfc(1/np.sqrt(2))*np.sqrt(np.e)+np.pi+2)**(-1/2)

def selu(z, scale=alpha_0_1, alpha=scale_0_1):
    return scale * tf.where(z >= 0.0, z, alpha * tf.nn.elu(z))

In [17]:
validate_x.shape

(100185, 928)

In [35]:
import tensorflow as tf
from functools import partial

reset_graph()

num_examples, n_inputs  = train_x.shape
n_hidden1 = 100 
n_hidden2 = 100 
n_outputs = 2 
learning_rate = 0.01
n_epochs=30
batch_size=64
batch_norm_momentum = 0.9


X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int32, shape=(None), name="y")
training = tf.placeholder_with_default(False, shape=(), name='training')

with tf.name_scope("dnn"):
    he_init = tf.variance_scaling_initializer()

    my_batch_norm_layer = partial(
            tf.layers.batch_normalization,
            training=training,
            momentum=batch_norm_momentum)

    my_dense_layer = partial(
            tf.layers.dense,
            kernel_initializer=he_init)

    hidden1 = my_dense_layer(X, n_hidden1, name="hidden1")
    bn1 = tf.nn.elu(my_batch_norm_layer(hidden1))
    hidden2 = my_dense_layer(bn1, n_hidden2, name="hidden2")
    bn2 = tf.nn.elu(my_batch_norm_layer(hidden2))
    logits_before_bn = my_dense_layer(bn2, n_outputs, name="outputs")
    logits = my_batch_norm_layer(logits_before_bn)

with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")

with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)

with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
    
init = tf.global_variables_initializer()
saver = tf.train.Saver()

extra_update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)

with tf.Session() as sess: 
    init.run() 
    for epoch in range (n_epochs): 
        for X_batch, y_batch in shuffle_batch(train_x, train_y, batch_size): 
            #X_batch_scaled = (X_batch - means) / stds
            sess.run([training_op, extra_update_ops], feed_dict= {training: True, X: X_batch, y: y_batch } ) 
        if epoch % 5 == 0:
            accuracy_test = accuracy.eval(feed_dict={X: test_x, y: test_y } ) 
            accuracy_val = accuracy.eval(feed_dict={X: validate_x, y: validate_y } ) 
            print (epoch, "TEST ACC: ", accuracy_test,"VAL ACC: ", accuracy_val ) 
    save_path = saver.save(sess, "./models/my_model_final.ckpt" ) 


0 TEST ACC:  0.9407881 VAL ACC:  0.9401008


KeyboardInterrupt: 

In [36]:
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.exceptions import NotFittedError

class DNNClassifier(BaseEstimator, ClassifierMixin):
    def __init__(self, n_hidden_layers=2, n_neurons=100, optimizer_class=tf.train.AdamOptimizer,
                 learning_rate=0.01, batch_size=20, activation=tf.nn.elu, initializer=he_init,
                 batch_norm_momentum=None, dropout_rate=None, random_state=None):
        """Initialize the DNNClassifier by simply storing all the hyperparameters."""
        self.n_hidden_layers = n_hidden_layers
        self.n_neurons = n_neurons
        self.optimizer_class = optimizer_class
        self.learning_rate = learning_rate
        self.batch_size = batch_size
        self.activation = activation
        self.initializer = initializer
        self.batch_norm_momentum = batch_norm_momentum
        self.dropout_rate = dropout_rate
        self.random_state = random_state
        self._session = None

    def _dnn(self, inputs):
        """Build the hidden layers, with support for batch normalization and dropout."""
        for layer in range(self.n_hidden_layers):
            if self.dropout_rate:
                inputs = tf.layers.dropout(inputs, self.dropout_rate, training=self._training)
            inputs = tf.layers.dense(inputs, self.n_neurons,
                                     kernel_initializer=self.initializer,
                                     name="hidden%d" % (layer + 1))
            if self.batch_norm_momentum:
                inputs = tf.layers.batch_normalization(inputs, momentum=self.batch_norm_momentum,
                                                       training=self._training)
            inputs = self.activation(inputs, name="hidden%d_out" % (layer + 1))
        return inputs

    def _build_graph(self, n_inputs, n_outputs):
        """Build the same model as earlier"""
        if self.random_state is not None:
            tf.set_random_seed(self.random_state)
            np.random.seed(self.random_state)

        X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
        y = tf.placeholder(tf.int32, shape=(None), name="y")

        if self.batch_norm_momentum or self.dropout_rate:
            self._training = tf.placeholder_with_default(False, shape=(), name='training')
        else:
            self._training = None

        dnn_outputs = self._dnn(X)

        logits = tf.layers.dense(dnn_outputs, n_outputs, kernel_initializer=he_init, name="logits")
        Y_proba = tf.nn.softmax(logits, name="Y_proba")

        xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y,
                                                                  logits=logits)
        loss = tf.reduce_mean(xentropy, name="loss")

        optimizer = self.optimizer_class(learning_rate=self.learning_rate)
        training_op = optimizer.minimize(loss)

        correct = tf.nn.in_top_k(logits, y, 1)
        accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")

        init = tf.global_variables_initializer()
        saver = tf.train.Saver()

        # Make the important operations available easily through instance variables
        self._X, self._y = X, y
        self._Y_proba, self._loss = Y_proba, loss
        self._training_op, self._accuracy = training_op, accuracy
        self._init, self._saver = init, saver

    def close_session(self):
        if self._session:
            self._session.close()

    def _get_model_params(self):
        """Get all variable values (used for early stopping, faster than saving to disk)"""
        with self._graph.as_default():
            gvars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES)
        return {gvar.op.name: value for gvar, value in zip(gvars, self._session.run(gvars))}

    def _restore_model_params(self, model_params):
        """Set all variables to the given values (for early stopping, faster than loading from disk)"""
        gvar_names = list(model_params.keys())
        assign_ops = {gvar_name: self._graph.get_operation_by_name(gvar_name + "/Assign")
                      for gvar_name in gvar_names}
        init_values = {gvar_name: assign_op.inputs[1] for gvar_name, assign_op in assign_ops.items()}
        feed_dict = {init_values[gvar_name]: model_params[gvar_name] for gvar_name in gvar_names}
        self._session.run(assign_ops, feed_dict=feed_dict)

    def fit(self, X, y, n_epochs=100, X_valid=None, y_valid=None):
        """Fit the model to the training set. If X_valid and y_valid are provided, use early stopping."""
        self.close_session()

        # infer n_inputs and n_outputs from the training set.
        n_inputs = X.shape[1]
        self.classes_ = np.unique(y)
        n_outputs = len(self.classes_)
        
        self._graph = tf.Graph()
        with self._graph.as_default():
            self._build_graph(n_inputs, n_outputs)
            # extra ops for batch normalization
            extra_update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)

        # needed in case of early stopping
        max_checks_without_progress = 20
        checks_without_progress = 0
        best_loss = np.infty
        best_params = None
        
        # Now train the model!
        self._session = tf.Session(graph=self._graph)
        with self._session.as_default() as sess:
            self._init.run()
            for epoch in range(n_epochs):
                rnd_idx = np.random.permutation(len(X))
                for rnd_indices in np.array_split(rnd_idx, len(X) // self.batch_size):
                    X_batch, y_batch = X[rnd_indices], y[rnd_indices]
                    feed_dict = {self._X: X_batch, self._y: y_batch}
                    if self._training is not None:
                        feed_dict[self._training] = True
                    sess.run(self._training_op, feed_dict=feed_dict)
                    if extra_update_ops:
                        sess.run(extra_update_ops, feed_dict=feed_dict)
                if X_valid is not None and y_valid is not None:
                    loss_val, acc_val = sess.run([self._loss, self._accuracy],
                                                 feed_dict={self._X: X_valid,
                                                            self._y: y_valid})
                    if loss_val < best_loss:
                        best_params = self._get_model_params()
                        best_loss = loss_val
                        checks_without_progress = 0
                    else:
                        checks_without_progress += 1
                    print("{}\tValidation loss: {:.6f}\tBest loss: {:.6f}\tAccuracy: {:.2f}%".format(
                        epoch, loss_val, best_loss, acc_val * 100))
                    if checks_without_progress > max_checks_without_progress:
                        print("Early stopping!")
                        break
                else:
                    loss_train, acc_train = sess.run([self._loss, self._accuracy],
                                                     feed_dict={self._X: X_batch,
                                                                self._y: y_batch})
                    print("{}\tLast training batch loss: {:.6f}\tAccuracy: {:.2f}%".format(
                        epoch, loss_train, acc_train * 100))
            # If we used early stopping then rollback to the best model found
            if best_params:
                self._restore_model_params(best_params)
            return self

    def predict_proba(self, X):
        if not self._session:
            raise NotFittedError("This %s instance is not fitted yet" % self.__class__.__name__)
        with self._session.as_default() as sess:
            return self._Y_proba.eval(feed_dict={self._X: X})

    def predict(self, X):
        class_indices = np.argmax(self.predict_proba(X), axis=1)
        return np.array([[self.classes_[class_index]]
                         for class_index in class_indices], np.int32)

    def save(self, path):
        self._saver.save(self._session, path)

In [37]:
dnn_clf = DNNClassifier(random_state=42)
dnn_clf.fit(train_x, train_y, n_epochs=10, X_valid=test_x, y_valid=test_y)

0	Validation loss: 0.364093	Best loss: 0.364093	Accuracy: 89.69%


KeyboardInterrupt: 

In [25]:
import tensorflow as tf
reset_graph()
restore_saver  = tf.train.import_meta_graph("models/model_tf.ckpt.meta")

X = tf.get_default_graph().get_tensor_by_name("X:0")
y = tf.get_default_graph().get_tensor_by_name("y:0")
loss = tf.get_default_graph().get_tensor_by_name("loss:0")
Y_proba = tf.get_default_graph().get_tensor_by_name("Y_proba:0")
logits = Y_proba.op.inputs[0]
accuracy = tf.get_default_graph().get_tensor_by_name("accuracy:0")

with tf.Session() as sess:
    restore_saver.restore(sess, tf.train.latest_checkpoint("models/"))
    #logits_output = sess.run(Y_proba.eval, feed_dict={X: validate_x})
    logits_output = Y_proba.eval(feed_dict={X: pred_x})

INFO:tensorflow:Restoring parameters from models/model_tf.ckpt


In [29]:
descr=df_pred[cols_descr]
pd.concat([descr,pd.DataFrame(logits_output[:,1], columns=['prob'])], axis=1)

,rid,course,marketTime,horseName,runners,decimalPrice,res_win,res_place,prob
0,36,Gulfstream Park,19:25,Sequaya,9,0.038462,0,0,3.820406e-07
1,36,Gulfstream Park,19:25,Prairie Wings,9,0.266667,0,0,1.329001e-02
2,36,Gulfstream Park,19:25,Pretty Darling,9,0.029412,0,0,2.382010e-02
3,36,Gulfstream Park,19:25,Risk Model,9,0.142857,0,0,4.140392e-03
4,36,Gulfstream Park,19:25,Primacy,9,0.523560,0,0,1.964036e-01
...,...,...,...,...,...,...,...,...,...
1094,12691,Tampa Bay Downs,21:00,Uncle Bud,19,0.142857,0,0,6.460781e-03
1095,12691,Tampa Bay Downs,21:00,River Hawk,19,0.052632,0,0,5.165096e-03
1096,12691,Tampa Bay Downs,21:00,Stuck Up,19,0.029412,0,0,5.881478e-04
1097,12691,Tampa Bay Downs,21:00,Summer Assault,19,0.546448,0,0,1.944117e-01


In [40]:
descr=pd.concat([df_pred[cols_descr],pd.DataFrame(logits_output[:,1], columns=['prob_tf'])], axis=1)
descr['probsum']=descr.groupby(['rid'])['prob_tf'].transform('sum')
descr['prob_tf']=descr['prob_tf']/descr['probsum']
descr['diff']=(descr['prob_tf']-descr['decimalPrice'])*10
descr['odds']=1/descr['decimalPrice']
descr['C']=1
descr.to_csv('data/today_tf.csv')

In [41]:
from catboost import CatBoostClassifier
model=CatBoostClassifier()
model.load_model('models/horses (8).cbm')
predv=model.predict(pred_x)
probv=model.predict_proba(pred_x)[:,1]

In [42]:
descr=pd.concat([df_pred[cols_descr],pd.DataFrame(probv, columns=['prob_cb'])], axis=1)
descr['probsum']=descr.groupby(['rid'])['prob_cb'].transform('sum')
descr['prob_cb']=descr['prob_cb']/descr['probsum']
descr['diff']=(descr['prob_cb']-descr['decimalPrice'])*10
descr['odds']=1/descr['decimalPrice']
descr['C']=1
descr.to_csv('data/today_cb.csv')

In [35]:
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score,  f1_score, log_loss
from sklearn.ensemble import GradientBoostingClassifier
model=GradientBoostingClassifier()
model=model.fit(test_x, test_y)
#joblib.dump(model, fn)
pred=model.predict(test_x)
prob=model.predict_proba(test_x)[:,1]
acc=accuracy_score(test_y, pred)
loss=log_loss(test_y,prob)

predv=model.predict(validate_x)
probv=model.predict_proba(validate_x)[:,1]
accv=accuracy_score(validate_y, predv)
lossv=log_loss(validate_y,probv)
print('GBC: test {:0.4f}, logloss:{:0.4f}; validate {:0.4f}, logloss:{:0.4f};'.format(acc,loss,accv,lossv))

probv=model.predict_proba(pred_x)[:,1]
descr=pd.concat([df_pred[['date']+cols_descr],pd.DataFrame(probv, columns=['prob_gbc'])], axis=1)
descr['probsum']=descr.groupby(['rid'])['prob_gbc'].transform('sum')
descr['prob']=descr['prob_gbc']/descr['probsum']
descr['diff']=(descr['prob_gbc']-descr['decimalPrice'])*10
descr['odds']=1/descr['decimalPrice']
descr['C']=1
descr.to_csv('data/today_gbc.csv')

NameError: name 'accuracy_score' is not defined